In [1]:

import argparse
import numpy as np
from data_loader import load_data
import logging
np.random.seed(2022)
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:

import numpy as np
import torch
import logging
from model import RippleNet
from sklearn.metrics import roc_auc_score, f1_score
from train import train, _get_user_record, get_feed_dict

import multiprocessing


usage: ipykernel_launcher.py [-h] [--dataset DATASET] [--dim DIM]
                             [--n_hop N_HOP] [--kge_weight KGE_WEIGHT]
                             [--l2_weight L2_WEIGHT] [--lr LR]
                             [--batch_size BATCH_SIZE] [--n_epoch N_EPOCH]
                             [--n_memory N_MEMORY]
                             [--item_update_mode ITEM_UPDATE_MODE]
                             [--using_all_hops USING_ALL_HOPS]
                             [--show_topk SHOW_TOPK] [--gpu_id GPU_ID]
                             [--Ks [KS]] [--test_flag [TEST_FLAG]]
                             [--use_cuda USE_CUDA] [--save SAVE]
                             [--out_dir OUT_DIR]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"7b4191c7-eaf4-4d8d-99e2-2d0919d637ef" --shell=9007 --transport="tcp" --iopub=9009 --f=/tmp/tmp-189298ok1TtuGjdngU.json


SystemExit: 2

In [ ]:
args = argparse.Namespace()
args.dataset = "naver-toy"
args.dim=32
args.n_hop=2
args.kge_weight=0.01
args.l2_weight=1e-7
args.lr = 0.02
args.batch_size = 256
args.n_epoch = 10
args.n_memory=32
args.item_update_mode="plus_transform"
args.using_all_hops=True
args.use_cuda = True
args.show_topk =True
args.gpu_id=0
args.cuda=True
args.Ks=[20,40,60,80,100]
args.test_flag = "part"


In [ ]:
Ks = args.Ks
BATCH_SIZE = args.batch_size
batch_test_flag = True
cores = multiprocessing.cpu_count() // 2
device = torch.device("cuda")

In [ ]:
data_info = load_data(args)

In [ ]:
show_loss = False

# Train Function

In [ ]:
train_data = data_info[0]
eval_data = data_info[1]
test_data = data_info[2]
n_entity = data_info[3]
n_relation = data_info[4]
ripple_set = data_info[5]

model = RippleNet(args, n_entity, n_relation)
if args.use_cuda:
    model.cuda()

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    args.lr,
)


In [ ]:
n_entity

In [ ]:
from tqdm import tqdm

In [ ]:
for step in tqdm(range(args.n_epoch)):
    np.random.shuffle(train_data)
    start = 0
    return_dict = model(*get_feed_dict(args, model, train_data, ripple_set, start, start + args.batch_size))
    loss = return_dict["loss"]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
start =0
items, labels, memories_h, memories_r, memories_t = get_feed_dict(args, model, train_data, ripple_set, start, start + args.batch_size)

In [ ]:
memories_t[0].shape # (batch_size, dim)

In [ ]:
train_user_set = _get_user_record(train_data,True)

In [ ]:
test_user_set = _get_user_record(test_data, True)

In [ ]:
user_dict = {
    "train_user_set" : train_user_set,
    "test_user_set" : test_user_set
}

In [ ]:
len(train_user_set.keys())

In [ ]:
len(test_user_set.keys())

In [ ]:
all_data = np.vstack([train_data, test_data,eval_data])


In [ ]:
def _get_item_record(data):
    item_dict = dict()
    for rating in data:
        user = rating[0]
        item = rating[1]
        label = rating[2]

        if label == 1:
            if item not in item_dict:
                item_dict[item] = set()
            item_dict[item].add(user)

    return item_dict

In [ ]:
all_item_set = _get_item_record(all_data)
all_user_set = _get_user_record(all_data, True)


In [ ]:
len(all_item_set.keys())

In [ ]:
n_items = len(all_user_set.keys())
n_users = len(all_item_set.keys())
n_nodes = n_entity + n_users

In [ ]:
n_params ={
    "n_items" : n_items,
    "n_users" : n_users,
    "n_entities" : n_entity,
    "n_relations" : n_relation,
    "n_nodes" : n_nodes
}

In [ ]:
len(model.o_list)

In [ ]:
from metrics import *
# from .parser import parse_args
import multiprocessing
import heapq
from time import time

def ranklist_by_heapq(user_pos_test, test_items, rating, Ks):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = 0.
    return r, auc

def get_auc(item_score, user_pos_test):
    item_score = sorted(item_score.items(), key=lambda kv: kv[1])
    item_score.reverse()
    item_sort = [x[0] for x in item_score]
    posterior = [x[1] for x in item_score]

    r = []
    for i in item_sort:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = AUC(ground_truth=r, prediction=posterior)
    return auc

def ranklist_by_sorted(user_pos_test, test_items, rating, Ks):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = get_auc(item_score, user_pos_test)
    return r, auc

def get_performance(user_pos_test, r, auc, Ks):
    precision, recall, ndcg, hit_ratio = [], [], [], []

    for K in Ks:
        precision.append(precision_at_k(r, K))
        recall.append(recall_at_k(r, K, len(user_pos_test)))
        ndcg.append(ndcg_at_k(r, K, user_pos_test))
        hit_ratio.append(hit_at_k(r, K))

    return {'recall': np.array(recall), 'precision': np.array(precision),
            'ndcg': np.array(ndcg), 'hit_ratio': np.array(hit_ratio), 'auc': auc}


def test_one_user(x):
    test_flag = args.test_flag
    # user u's ratings for user u
    rating = x[0] # rate_batch
    
    # uid
    u = x[1]
    # user u's items in the training set
    try:
        training_items = train_user_set[u]
    except Exception:
        training_items = []
    # user u's items in the test set
    user_pos_test = test_user_set[u]
    # print(user_pos_test)
    all_items = set(range(0, n_entity))

    test_items = list(all_items - set(training_items))
    # print(len(test_items))
    if test_flag == 'part':
        r, auc = ranklist_by_heapq(user_pos_test, test_items, rating, Ks)
    else:
        r, auc = ranklist_by_sorted(user_pos_test, test_items, rating, Ks)

    return get_performance(user_pos_test, r, auc, Ks)

# def test(model, user_dict, n_params):
#     result = {'precision': np.zeros(len(Ks)),
#               'recall': np.zeros(len(Ks)),
#               'ndcg': np.zeros(len(Ks)),
#               'hit_ratio': np.zeros(len(Ks)),
#               'auc': 0.}

#     global n_users, n_items
#     n_items = n_params['n_items']
#     n_users = n_params['n_users']

#     global train_user_set, test_user_set
#     train_user_set = user_dict['train_user_set']
#     test_user_set = user_dict['test_user_set']

#     pool = multiprocessing.Pool(cores)

#     u_batch_size = BATCH_SIZE
#     i_batch_size = BATCH_SIZE

#     test_users = list(test_user_set.keys())
#     n_test_users = len(test_users)
#     n_user_batchs = n_test_users // u_batch_size + 1

#     count = 0

    
    
#     for u_batch_id in range(n_user_batchs):
#         start = u_batch_id * u_batch_size
#         end = (u_batch_id + 1) * u_batch_size

#         user_list_batch = test_users[start: end]
#         user_batch = torch.LongTensor(np.array(user_list_batch)).to(device)
#         u_g_embeddings = model.get_user_embeddings()

#         if batch_test_flag:
#             # batch-item test
#             n_item_batchs = n_items // i_batch_size + 1
#             rate_batch = np.zeros(shape=(len(user_batch), n_items))

#             i_count = 0
#             for i_batch_id in range(n_item_batchs):
#                 i_start = i_batch_id * i_batch_size
#                 i_end = min((i_batch_id + 1) * i_batch_size, n_items)

#                 item_batch = torch.LongTensor(np.array(range(i_start, i_end))).view(i_end-i_start).to(device)
#                 i_g_embddings = model.entity_emb(item_batch)

#                 i_rate_batch = model.rating(u_g_embeddings, i_g_embddings).detach().cpu()

#                 rate_batch[:, i_start: i_end] = i_rate_batch
#                 i_count += i_rate_batch.shape[1]

#             assert i_count == n_items
#         else:
#             # all-item test
#             item_batch = torch.LongTensor(np.array(range(0, n_items))).view(n_items, -1).to(device)
#             # i_g_embddings = entity_gcn_emb[item_batch]
#             # rate_batch = model.rating(u_g_embeddings, i_g_embddings).detach().cpu()
#             rate_batch = model.rating(u_g_embeddings, i_g_embddings).detach().cpu()

#         user_batch_rating_uid = zip(rate_batch, user_list_batch)
#         batch_result = pool.map(test_one_user, user_batch_rating_uid)
#         count += len(batch_result)

#         for re in batch_result:
#             result['precision'] += re['precision']/n_test_users
#             result['recall'] += re['recall']/n_test_users
#             result['ndcg'] += re['ndcg']/n_test_users
#             result['hit_ratio'] += re['hit_ratio']/n_test_users
#             result['auc'] += re['auc']/n_test_users

#     assert count == n_test_users
#     pool.close()
#     return result


In [ ]:
len(test_user_set[8169])

In [ ]:
n_params

In [ ]:
result = {'precision': np.zeros(len(Ks)),
              'recall': np.zeros(len(Ks)),
              'ndcg': np.zeros(len(Ks)),
              'hit_ratio': np.zeros(len(Ks)),
              'auc': 0.}

In [ ]:
n_items = n_params["n_items"]
n_users = n_params["n_users"]

train_user_set = user_dict["train_user_set"]
test_user_set = user_dict["test_user_set"]

In [ ]:
pool = multiprocessing.Pool(cores)
u_batch_size = BATCH_SIZE
i_batch_size = BATCH_SIZE

test_users = list(test_user_set.keys())
n_test_users = len(test_users)
n_user_batchs = n_test_users // u_batch_size +1


In [ ]:
11032 - (11032 // batch_size) * batch_size


In [ ]:
batch_size

In [ ]:
u_count = 0
for u_batch_id in range(n_user_batchs):
    start = u_batch_id * u_batch_size
    end = min((u_batch_id+1)*u_batch_size, n_test_users)

    user_list_batch = test_users[start:end]
    user_batch = torch.LongTensor(np.array(user_list_batch)).to(device)
    user_emb = model.get_user_embeddings(*get_feed_dict(args, model, test_data, ripple_set, start, end))
    
    n_items_batch = n_entity // i_batch_size + 1
    rate_batch = np.zeros(shape=(len(user_batch), n_entity))

    i_count = 0

    for i_batch_id in range(n_items_batch):
        i_start = i_batch_id * i_batch_size
        i_end = min((i_batch_id+1)*i_batch_size, n_entity)

        item_batch = torch.LongTensor(np.array(range(i_start, i_end))).view(i_end-i_start).to(device)
        item_emb = model.entity_emb(item_batch)

        i_rate_batch = model.rating(user_emb, item_emb).detach().cpu()
        rate_batch[:, i_start:i_end] = i_rate_batch
        # print(f"rate_batch.shape: {rate_batch[:, i_start:i_end].shape}")
        # print(f"i_rate_batch.shape: {i_rate_batch.shape}")
        print(f"len(user_batch) : {len(user_batch)}")
    # print(rate_batch.shape, len(user_list_batch))
    user_batch_rating_uid = zip(rate_batch, user_list_batch)
    # batch_result = list()
    batch_result = pool.map(test_one_user, user_batch_rating_uid)
    for re in batch_result:

        result['precision'] += re['precision']/n_test_users
        result['recall'] += re['recall']/n_test_users
        result['ndcg'] += re['ndcg']/n_test_users
        result['hit_ratio'] += re['hit_ratio']/n_test_users
        result['auc'] += re['auc']/n_test_users

In [ ]:
result

In [ ]:
n_user_batchs * 256 + 24